<a href="https://colab.research.google.com/github/Jatchen/Acorn_Practicum/blob/main/0115_Acorn_%E8%B7%91%E4%B8%80%E5%A4%A9%E7%9A%84%E5%85%A8%E9%83%A8%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import ast  # Import the ast module for safe literal evaluation
import re  # Import the 're' module

In [36]:
original_2_df = pd.read_csv("/content/drive/MyDrive/result_2_df.csv")
# 用于跑改变input进行对比
result_2_df = original_2_df.head(int(len(original_2_df) * 0.4))

In [26]:
pip install pulp

In [27]:
import pulp

In [37]:
# Parameters
p = 3  # Minimum number of potential buyers for each property
q = 40  # Maximum number of times a buyer is listed in the property recommendations

# Clear previous variables by creating a new problem object
prob = pulp.LpProblem("PropertyAssignmentProblem", pulp.LpMaximize)

In [38]:
# Decision variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in result_2_df.property_id.unique() for j in result_2_df.customer_id.unique()), cat='Binary')

# Objective function
prob += pulp.lpSum([result_2_df.loc[(result_2_df.property_id == i) & (result_2_df.customer_id == j), 'rating'].values[0] * x[(i, j)]
                    if not result_2_df.loc[(result_2_df.property_id == i) & (result_2_df.customer_id == j), 'rating'].empty
                    else 0
                    for i in result_2_df.property_id.unique() for j in result_2_df.customer_id.unique()])


In [30]:
# Constraints
for i in result_2_df.property_id.unique():
    prob += pulp.lpSum([x[(i, j)] for j in result_2_df.customer_id.unique()]) >= p

for j in result_2_df.customer_id.unique():
    prob += pulp.lpSum([x[(i, j)] for i in result_2_df.property_id.unique()]) <= q

# Decision variable x: relax the binary constraint - make it a category integer: Add constraints x >= 0, x <= 1
# Or even Decision variable x:  relax binary constraint more and let x be a floating point number: Add constraints x>=0, x <=1
# how to see if sucess or not: Success if:  faster AND the xij output variables are all 0 or 1;Failure:  xij = 0.75 or some other nonbinary number

In [31]:
# Solve the problem
prob.solve()

# Print the status of the solution
print("Status:", pulp.LpStatus[prob.status])

# Print the total utility
print("Total Utility of Assignments = ", pulp.value(prob.objective))

Status: Infeasible
Total Utility of Assignments =  2207.723899036646


In [32]:
# 新的数据导出方式的尝试
# Create a list to store the data
new_data = []

# Extract information from all variables
for v in prob.variables():
    buyer_id, property_id = map(int, re.findall(r'\d+', v.name))
    xij_value = v.varValue
    new_data.append((property_id, buyer_id, xij_value))

# Create a new DataFrame
new_df = pd.DataFrame(new_data, columns=['property_id', 'customer_id', 'Xij_value'])

# 交换 customer_id 和 property_id 列
new_df['customer_id'], new_df['property_id'] = new_df['property_id'], new_df['customer_id']

# merge rating to result
# 确保 'property_id' 和 'customer_id' 列在两个 DataFrame 中都是相同的数据类型
new_df['property_id'] = new_df['property_id'].astype('int64')
new_df['customer_id'] = new_df['customer_id'].astype('int64')
result_2_df['property_id'] = result_2_df['property_id'].astype('int64')
result_2_df['customer_id'] = result_2_df['customer_id'].astype('int64')

merged_df = pd.merge(new_df, original_2_df, on=['property_id', 'customer_id'], how='inner')
print(merged_df)

       property_id  customer_id  Xij_value  property_profile_id    rating
0           100033      2330137        0.0                   15  1.552531
1           100033      3384124        1.0                   15  1.250265
2           100033      3384420        0.0                   15  1.198251
3           100033      3384504        0.0                   15  1.132391
4           100033      3384648        0.0                   15  1.231637
...            ...          ...        ...                  ...       ...
17327        99980       463685        0.0                   15  0.976388
17328        99980      4983095        0.0                   15  1.622355
17329        99980      6655588        0.0                   15  1.148170
17330        99980      7217388        0.0                   15  1.042675
17331        99980      7252559        0.0                   15  1.198251

[17332 rows x 5 columns]


<ipython-input-32-68c55bd89fa9>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_2_df['property_id'] = result_2_df['property_id'].astype('int64')
<ipython-input-32-68c55bd89fa9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_2_df['customer_id'] = result_2_df['customer_id'].astype('int64')


In [33]:
# 直接得到总分
total_rating = merged_df[merged_df['Xij_value'] == 1]['rating'].sum()
print(total_rating)

2231.6573081314564


In [35]:
#检验是否实现了小于40的分配
import pandas as pd

# 读取数据
df = merged_df

# 过滤 Xij=1 的数据
filtered_df = df[df['Xij_value'] == 1]

# 统计每个 customer_id 对应的不同 property_id 数量
result = filtered_df.groupby('customer_id')['property_id'].nunique().reset_index()

# 打印结果
print(result)


    customer_id  property_id
0        434187           15
1        438060           34
2        440738           24
3        445168           25
4        446153            1
..          ...          ...
68      7117168           36
69      7148259           27
70      7252559           46
71      7626260           38
72      7683672            1

[73 rows x 2 columns]


In [ ]:
#下载结果
merged_df.to_csv('merged_2_df.csv', index=False)

In [ ]:
from google.colab import files

In [ ]:
files.download('merged_2_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 跑不同的input data并且和random进行对比